# Tabula
It's only detecting tables in PDF.

`pip install tabula-py`

- ! Not for Account statement - not getting formatted data for amounts
- ! Not for Payslip - not getting formatted data

In [2]:
import tabula

tables = tabula.read_pdf('../assets/Vehicle Quotation.pdf', pages=1)
df = tables[0]
print(df)

print('---------\n')
print(f"VIN: {df['VIN'][0]}")
print(f"make Code: {df['Make Code']}")
print(f"Fuel Type: {df['Fuel Type']}")
print(f"Manufacturing Year: {df['Mfg Year']}")
print(f"Engine Number: {df['Engine Number']}")

                 VIN                  Make Code Fuel Type  Mfg Year  \
0  MEXBRFPB4NG010495  SLAVIA STYLE 1.5 TSI MT-P    PETROL      2022   

  Engine Number  
0    DUM A16928  
---------

VIN: MEXBRFPB4NG010495
make Code: 0    SLAVIA STYLE 1.5 TSI MT-P
Name: Make Code, dtype: object
Fuel Type: 0    PETROL
Name: Fuel Type, dtype: object
Manufacturing Year: 0    2022
Name: Mfg Year, dtype: int64
Engine Number: 0    DUM A16928
Name: Engine Number, dtype: object


In [43]:
import tabula

tables = tabula.read_pdf("../assets/Account Statement 3.pdf", pages=1)
print(tables)


[]


# Pdfminer

`pip install pdfminer.six`

- !Not working for acc. stmt. 3

In [3]:
import re
from pdfminer.high_level import extract_pages, extract_text

In [120]:
text = extract_text('../assets/Payslip3.pdf')

print(text)

Rumango Software and Consulting Services Private Limited

PAYSLIP FOR THE MONTH OF JUNE 2022

Emp Name

Bhagabati Panda

Date Of Joining

27 Apr 2022

PAN No:

Emp ID

31161

Date of Birth

10 Mar 1998

GSTIN No:

Designation

Trainee Engineer

Bank A/C No

110019862705

Bank Name

EHKPP9969J

33AAICR8073M1ZU

Gender

Male

UAN No.

101827547770

Branch Name

EPIP

Earnings

Basic Earned

HRA Earned

Conveyance Allowance Earned

Meal Allowance Earned

Children Education Allowance Earned

Medical Allowance Earned

Special Allowance Earned

Telephone Allowance Earned

LTA Earned

Incentives

Joining Bonus

Performance Bonus

Adjustment

Arrears

Hardship Allowance

Gross Earnings :

Adjustment

Leave Salary

Gross Earned

Arrears

Joining Bonus

Amount

Deductions

15,723.00

Sal. Deduction

6,289.00

Employee PF Contribution

1,600.00

PT

1,100.00

TDS

Amount

0.00

1,886.00

200.00

0.00

0.00

1,250.00

8,126.00

1,666.00

0.00

0.00

0.00

2,358.00

0.00

0.00

0.00

38,113.00 Tota

In [44]:
text = extract_text("../assets/Account Statement 3.pdf")

print(text)

Statement for A/c XXXXXXXX2705 for the period 19-Dec-2023 to 18-Jun-2024

Customer Id

XXXXXXX74

Name

Phone

BHAGABATI PRASAD PAN

+918249194704

Address
CO MANORANJAN PADA PLOT
NO  L  1929  PHASE  2  DUMUDUMA  HOUSING
BOARD COLONY BHUBANESWAR ODISHA

Branch Code

18017

Branch Name
CAMPUS

BHUBANESHWAR CET

IFSC Code

CNRB0018017

Address
CET CAMPUS KALINGANAGAR,
KALINGANAGAR  GHATIKIA,    BHUBANESWAR
ODISHA

Date

Particulars

Deposits

Withdrawals

Balance

Opening Balance

1,485.85

17-12-2023

17-12-2023

17-12-2023

17-12-2023

UPI/DR/335151494783/BALAKRI
SH/PYTM/**E0OY5@PAYTM/PA
YMENT
//AXLB3185E87920F489CA3CC
56FDEBBEA0E7/17/12/2023
19:14:16

Chq: 335151494783

UPI/DR/335109910094/VIJAYAL
AK/FDRL/**13629@FBPE/PAY
TO
B//AXL6A6FF1F1BC644CD3B08
F210398351687/17/12/2023
19:41:41

Chq: 335109910094

UPI/DR/335164486661/SOUBHA
GYA/PYTM/**LIDIZ@PAYTM/PA
YMENT
//AXLF938DB8B91D74D82B887
9031CE01E8A7/17/12/2023
19:43:11

Chq: 335164486661

UPI/DR/371708253997/SAGAR
PUJ/ICIC/**GAR10@YBL

# PyMuPDF & Pillow

In [24]:
import fitz # PyMuPDF
import PIL.Image # pillow
import io

In [25]:
pdf = fitz.open('../assets/Payslip.pdf')
counter=1
for i in range(len(pdf)):
    page = pdf[i]
    images = page.get_images()
    for image in images:
        base_img = pdf.extract_image(image[0])
        image_data = base_img['image']
        img = PIL.Image.open(io.BytesIO(image_data))
        extension = base_img['ext']
        img.save(open(f"image{counter}.{extension}", 'wb'))
        counter += 1

# PdfPlumber

### Payslip

In [79]:
import pdfplumber
import re

# Initialize the variables
payslip_data = ''
employer_name = ""
employee_name = ""
designation = ""
employment_start_date = ""
monthly_income = ""

with pdfplumber.open('../assets/Payslip.pdf') as pdf:
    # print(pdf.pages[0].extract_text())
    payslip_data = pdf.pages[0].extract_table()

    # Remove newlines from the text in the data
    for i in range(len(payslip_data)):
        payslip_data[i] = [
            item.replace("\n", " ") if item else item for item in payslip_data[i]
        ]

    # Iterate over each list in payslip_data to find the required details
    for item in payslip_data:
        print(item)

        # Extract employer name
        if item[0] and "PAYSLIP FOR THE MONTH OF" in item[0]:
            employer_name = item[0].split("\n")[0]

        # Extract employee name
        if "Emp Name" in item or "Employee Name" in item:
            emp_name_index = (
                item.index("Emp Name")
                if "Emp Name" in item
                else item.index("Employee Name")
            )
            employee_name = item[emp_name_index + 1]

        # Extract designation
        if "Designation" in item:
            designation_index = item.index("Designation")
            designation = item[designation_index + 1]

        # Extract employment start date
        if "Date Of Joining" in item or "Date of Joining" in item:
            employment_start_date_index = (
                item.index("Date Of Joining")
                if "Date Of Joining" in item
                else item.index("Date of Joining")
            )
            employment_start_date = item[employment_start_date_index + 2]

        # Extract monthly income
        if any("Net Pay:" in str(sub_item) for sub_item in item):
            for sub_item in item:
                if isinstance(sub_item, str) and "Net Pay:" in sub_item:
                    monthly_income_match = re.search(r"Rs\.\s([\d,]+.\d+)", sub_item)
                    if monthly_income_match:
                        monthly_income = monthly_income_match.group(1)

print('\n-----------------')

# Print the extracted details
print("Employer Name:", employer_name)
print("Employee Name:", employee_name)
print("Designation:", designation)
print("Employment Start Date:", employment_start_date)
print("Monthly Income:", monthly_income)

['Rumango Software and Consulting Services Private Limited PAYSLIP FOR THE MONTH OF DECEMBER 2023', None, None, None, None, None, None, None, None]
['Emp Name', 'Abhilash Kar', None, 'Date Of Joining', '27 Jun 2022', 'PAN No.', None, 'IMRPK7360G', None]
['Emp ID', '31174', None, 'Date of Birth', '16 May 2021', 'GSTIN No.', None, '33AAICR8073M1ZU', None]
['Designation', 'Associate Software Developer', None, 'Bank A/C No.', '50100652600311', 'Bank Name', None, 'HDFC Bank', None]
['Gender', 'Male', None, 'UAN No.', '101853131834', 'Branch Name', None, 'EPIP', None]
['', None, None, None, None, None, None, None, None]
['Earnings', None, 'Amount', 'YTD', 'Deductions', None, 'Amount', None, 'YTD']
['Basic Earned HRA Earned Conveyance Allowance Earned Meal Allowance Earned Children Education Allowance Earned Medical Allowance Earned Special Allowance Earned Telephone Allowance Earned LTA Earned Incentives Joining Bonus Performance Bonus Adjustment Arrears Hardship Allowance', None, '15,739.00

### Vehicle Quotation

In [43]:
with pdfplumber.open("../assets/Vehicle Quotation.pdf") as pdf:
    data = pdf.pages[0].extract_table()
    for info in data:
        print(info)

['SKODA AUTO VOLKSWAGEN INDIA PRIVATE LIMITED\n(Formally known as Volkswagen India Private Limited)\nPlot No A-1 /1, SHENDRA Five Star Industrial Area\nMIDC, Aurangabad - 431201. India\nSkoda Maintenance Package', None, None, None, None]
['', None, None, None, None]
['Dealership Name: RAJA GANESH MOTORS Issued date : 12-Aug-2022\nService and Maintenance Plan No : C-SM-IND663-BAS Sta\x00 Date:: 02-Jul-2022\nCe\x00i\x00cate No. :SMP9822248179 End Date: 31-Jul-2026\nSta\x00 Mileage: 0\nEnd Mileage: 63,000', None, None, None, None]
['', None, None, None, None]
['Customer Name: Mr. Ka\x00hikeyan Chandrasekaran\nAddress: A 305 Splendid Eternity Apa\x00ments\n87/3 Hoodi Main Road Hoodi Mahadevpura\nBangalore\nMobile No: (R) 9742311311,(M) 9742311311\nEmail ID: kmuka\x00hik@gmail.com', None, None, None, None]
['', None, None, None, None]
['VIN', 'Make Code', None, 'Fuel Type', 'Mfg Year Engine Number']
['MEXBRFPB4NG010495', 'SLAVIA STYLE 1.5 TSI MT-P', None, 'PETROL', '2022 DUM A16928']
['', N

### Account Statement

In [96]:
with pdfplumber.open("../assets/Account Statement.pdf") as pdf:
    text = pdf.pages[0].extract_text()
    print(text)

    print('---------')
    table_data = pdf.pages[0].extract_table()
    for info in table_data:
        print(info)

PageNo.:1
AccountBranch : EPIP
Address : NO120B,EPIPINDUSTRIALAREA,
OPPINORBITMALL,
WHITEFIELD,BANGALORE,
City : BENGALURU560066
MR PRAVEENRAJ
State : KARNATAKA
RUMANGOSOFTWARECONSULTINGAND Phoneno. : 18002026161
SERVICESPVTLTDAKSHAYTECHPARK ODLimit : 0.00
Currency : INR
.
Email : PRAVEEN89333@GMAIL.COM
BANGALORE560066 CustID : 251831048
KARNATAKAINDIA AccountNo : 50100648442963 OTHER
A/COpenDate : 10/08/2023
JOINTHOLDERS: AccountStatus : Regular
RTGS/NEFTIFSC: HDFC0003962 MICR:560240112
BranchCode : 3962 ProductCode:113
Nomination: NotRegistered
From : 01/11/2023 To : 30/04/2024 Statementof account
Date Narration Chq./Ref.No. ValueDt WithdrawalAmt. DepositAmt. ClosingBalance
01/11/23 UPI-GIGUHOSPITALITYPVT-BHARATPE.905001 0000330563501888 01/11/23 2,609.00 5,108.41
5821@FBPE-FDRL0001382-330563501888-PAYT
OGIGUHOSPIT
01/11/23 UPI-TEASPARK-PAYTMQR281005050101KWVY78R 0000330574617154 01/11/23 32.00 5,076.41
7H0BY@PAYTM-PYTM0123456-330574617154-UPI
01/11/23 UPI-ANOJJA MURUGAPPAN-ANOJJA.IA